<a href="https://colab.research.google.com/github/alphakilo11/Python/blob/main/MTG/221013_Deckgenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from typing_extensions import final
# https://github.com/MagicTheGathering/mtg-sdk-python
!pip install mtgsdk
from mtgsdk import Card
from mtgsdk import Set

def cardnames(cards):
  """print list of card names"""
  return list(i.name for i in cards)

def create_decklist(list_of_cards, setname, number_of_each_card, deckname):
  """ creates a decklist in Forge format (*.dck)"""
  decklist = '[metadata]\nName=' + deckname +'\n[Main]\n'
  for card in list_of_cards:
    decklist += str(number_of_each_card) + ' ' + card.name + '|' + setname + '\n'
  return decklist

def count_type(list_of_cards, cardtype):
  count = 0
  relevant_chars = len(cardtype)
  for card in list_of_cards:
    if cardtype in card.type:
      count += 1
  return count

def average_cmc(list_of_cards):
  sum = 0
  count = 0
  for card in list_of_cards:
    if 'Land' not in card.type:
      sum += card.cmc
      count += 1
  return sum / count

def required_lands(list_of_cards):
  """determine required number of lands according to https://strategy.channelfireball.com/all-strategy/home/how-many-lands-do-you-need-in-your-deck-an-updated-analysis/
  aforementioned formula is not really useful for creating variable size decks,  as I don't know how many cards a deck will eventually have"""
  #return len(list_of_cards) / 60 * (19.59 + 1.9 * average_cmc(list_of_cards))
  return (19.59 + 1.9 * average_cmc(list_of_cards))

def deckgenerator(setname, decksize, desired_non_land_cards, random_lands, debug=False):
  """
  This generator creates a number of 60-card mtg decks using all cards from a given set. The decks include lands for all five colors.
  The decks are stored as *.dck file to be used in Forge.
  Input:
  * Name of Set
  * how many cards should each deck contain (only works when random_lands=True)
  * desired number of non-land-cards in each deck
  * should a random number of copies form the landpool be added?
  currently the generated deck does not contain any lands from the provided set"""
  #IDEA use the Channel Fireball land requirment formula and try (or calculate if able) which decksize is closest to the ideal
  #BUG sometimes decks will have less than decksize cards (reproduce: 5ED)
  #BUG more than 4 copies of a non-standard land might be added to decks (chance ~ 3.5 %)
  #ENHANCE should have exactly one of each standard land (from same set preferred)
  #ENHANCE distribute set-lands evenly
  #ENHANCE Distribute CMC evenly
  #ENHANCE use card objects for lands
  #ENHANCE the program should determine the average CMC of each deck and adjust the number of lands accordingly https://strategy.channelfireball.com/all-strategy/home/how-many-lands-do-you-need-in-your-deck-an-updated-analysis/
  #CHECK do modal double-faced cards (MDFCs) appear twice in a decklist?
  import random
  from timeit import default_timer as timer

  start_time = timer()
  cards = Card.where(set=setname).all() # getting the cards from the API
  if len(cards) < 1:
    print("Fetching any cards failed.")
    return False
  #shuffle
  random.shuffle(cards)
  # Seperate land cards
  lands, non_lands = [], []
  for card in cards:
    if 'Land' not in card.type:
      non_lands.append(card)
    else:
      lands.append(card)

  # calculate number of decks
  number_of_decks = round(len(non_lands) / desired_non_land_cards)
  if number_of_decks == 0:
    print("too few cards to create a useful deck")
    return False

  # Seperate creature cards
  creatures, non_creatures = [], []
  for card in non_lands:
    if 'Creature' not in card.type:
      non_creatures.append(card)
    else:
      creatures.append(card)

  decks = {} # this is where the actual decks will be put into
  #constructing decknames and creating empty lists for each
  decknames = []
  for i in range(number_of_decks):
    name = setname + '_' + str(i) + '.dck'
    decknames.append(name)
    decks[name] = []

  card_types = [creatures, non_creatures]
  for typedeck in card_types:
    counter, checker, cursor = 0, 0, 0
    for card in typedeck:
      decks[decknames[cursor]].append(card)
      checker += 1
      cursor += 1
      if cursor > number_of_decks - 1:
        cursor = 0
    if checker != len(typedeck):
      print("Not all cards from the typedeck", typedeck, " (", len(typedeck), ") made it into the decks(", checker,").")

  # Build the landpool
  landpool = []
  with open('/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Manabase/mb_autogen.txt', 'r') as file:
    content = file.readline()[:-1]
    while content != '':
      landpool.append(content)
      content = file.readline()[:-1]
  del content
  # Convert the landpool to mtgsdk card objects
  final_landpool = []
  for land in landpool:
    final_landpool.append(Card.where(name=land[2:]).all()[0])
  if debug == True:
    print(f'Landpool: {[card.name for card in final_landpool]}')
  del landpool

  for name in decknames:
    writecache = create_decklist(decks[name], setname, 1, name[:-4])
    # add lands
    if random_lands == True:
      # add lands to decks
      while len(decks[name]) < decksize:
          land_card = random.choice(final_landpool)
          decks[name].append(land_card)
        # as the setname of the lands is usually different, I have to add them manually
          writecache += '1 ' + land_card.name + '\n'

    else:
      with open('/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Manabase/mb_autogen.txt', 'r') as file:
        land_template = file.readlines()
      counter = 0
      template_line = 0
      while counter < decksize - len(decks[name]):
        max_copies = land_temp # CONTINUE HERE!
        writecache += land_template[counter]
        counter += 1
      writecache += content

    filename = '/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Autogen/' + name
    with open(filename, 'w') as file:
      file.write(writecache)
    non_land_count = len(decks[name]) - count_type(decks[name], 'Land')
    print("Created", name, len(decks[name]), " cards.", count_type(decks[name], 'Creature'), "creatures.", count_type(decks[name], 'Land'), "lands. Average CMC:", round(average_cmc(decks[name]), 2)\
          , "Required lands:", round(required_lands(decks[name]), 2)) # these stats only use the data from card objects

  print("Dauer: ", timer() - start_time)
  return True

In [44]:
deckgenerator('5ED', 60, 36, random_lands=True, debug=True)

Landpool: ['Plains', 'Island', 'Swamp', 'Mountain', 'Karplusan Forest', 'Exotic Orchard', 'Reflecting Pool', 'Gateway Plaza', 'Archway Commons', 'Gemstone Mine']
Created 5ED_0.dck 60  cards. 16 creatures. 24 lands. Average CMC: 3.06 Required lands: 25.4
Created 5ED_1.dck 60  cards. 16 creatures. 24 lands. Average CMC: 2.53 Required lands: 24.39
Created 5ED_2.dck 60  cards. 16 creatures. 24 lands. Average CMC: 2.86 Required lands: 25.03
Created 5ED_3.dck 60  cards. 16 creatures. 24 lands. Average CMC: 3.08 Required lands: 25.45
Created 5ED_4.dck 60  cards. 16 creatures. 24 lands. Average CMC: 3.28 Required lands: 25.82
Created 5ED_5.dck 60  cards. 16 creatures. 25 lands. Average CMC: 2.66 Required lands: 24.64
Created 5ED_6.dck 60  cards. 16 creatures. 25 lands. Average CMC: 2.97 Required lands: 25.24
Created 5ED_7.dck 60  cards. 15 creatures. 26 lands. Average CMC: 2.97 Required lands: 25.23
Created 5ED_8.dck 60  cards. 15 creatures. 26 lands. Average CMC: 2.38 Required lands: 24.12
Cr

True

In [ ]:
# creating a dict of lands with the objective to get a list of card objects for the lands
landpool = []
with open('/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Manabase/mb_autogen.txt', 'r') as file:
  content = file.readline()[:-1]
  while content != '':
    landpool.append(content)
    content = file.readline()[:-1]
del content

land_dict = {}
for entry in landpool:
  land_dict[entry[2:]] = {"copies": entry[:1], "set": "KTK"}
land_dict["Exotic Orchard"]['set'] = "CON"
print(land_dict)


{'Plains': {'copies': '1', 'set': 'KTK'}, 'Island': {'copies': '1', 'set': 'KTK'}, 'Swamp': {'copies': '1', 'set': 'KTK'}, 'Mountain': {'copies': '1', 'set': 'KTK'}, 'Forest': {'copies': '1', 'set': 'KTK'}, 'Exotic Orchard': {'copies': '4', 'set': 'CON'}, 'Plaza of Heroes': {'copies': '1', 'set': 'KTK'}, 'Cascading Cataracts': {'copies': '4', 'set': 'KTK'}, 'Reflecting Pool': {'copies': '1', 'set': 'KTK'}, 'City of Brass': {'copies': '1', 'set': 'KTK'}, 'Mana Confluence': {'copies': '4', 'set': 'KTK'}, 'Fabled Passage': {'copies': '4', 'set': 'KTK'}}


In [ ]:
#what is p that more than 4 copies of a non-standard land are added to a deck
repetitions = 10000000
land_cards = 25
landpool = []

with open('/content/drive/MyDrive/Brettspiele&Co/Magic the Gathering/Decklists/Manabase/mb_autogen.txt', 'r') as file:
  content = file.readline()[:-1]
  while content != '':
    landpool.append(content)
    content = file.readline()[:-1]
del content

positives = 0
for i in range(repetitions):
  sample = []
  for i in range(land_cards):
    draw = random.choice(landpool)
    if draw not in landpool[:5]: # assuming that the first 5 entries are standard lands
      sample.append(draw)
  my_dict = {i:sample.count(i) for i in sample}
  for i in range(5, n + 1):
    if i in my_dict.values():
      #print(my_dict)
      positives += 1
      break
print(round(positives / repetitions * 100, 2), '%')

3.47 %


In [15]:
#example
test = Card.where(name='Ancient Ziggurat').all() # Changed .iter() to .all()
print(cardnames(test)) # print list of card names
print(test[0].types)
from pprint import pprint

# Access the first card's properties using indexing
if test:  # Check if the list is not empty
    pprint(vars(test[0]))  # print all properties of the first card
del test

['Ancient Ziggurat', 'Ancient Ziggurat', 'Ancient Ziggurat', 'Ancient Ziggurat', 'Ancient Ziggurat', 'Ancient Ziggurat']
['Land']
{'artist': 'John Avon',
 'border': None,
 'cmc': 0.0,
 'color_identity': None,
 'colors': None,
 'flavor': "Built in honor of Alara's creatures, the ziggurat vanished long "
           'ago. When Progenitus awakened, the temple emerged again.',
 'foreign_names': [{'flavor': 'Die Zikkurat, als Ehrenmal für Alaras Kreaturen '
                              'erbaut, verschwand vor langer Zeit. Als '
                              'Progenitus erwachte, tauchte auch der Tempel '
                              'wieder auf.',
                    'identifiers': {'multiverseId': 193244,
                                    'scryfallId': 'b0e914d7-f0a2-4eae-93bc-c23e74378c24'},
                    'imageUrl': 'http://gatherer.wizards.com/Handlers/Image.ashx?multiverseid=193244&type=card',
                    'language': 'German',
                    'multiverseid': 193244

In [ ]:
boosterpack = Set.generate_booster('ktk')
print(list(i.name for i in boosterpack))

['Mantis Rider', 'Winterflame', 'Pine Walker', 'Seeker of the Way', 'Highland Game', 'Cancel', 'Plains', 'Temur Banner', 'Shatter', 'Longshot Squad', 'Weave Fate', "Archers' Parapet", 'Debilitating Injury', 'Plains']


In [ ]:
# possible phase class characteristics: phases end when all players pass with stack empty
# possible both chars: END: emtpy mana, "until end of" effects expire BEGIN: "until" effects expire, "at the beginning of" triggers
#HeadsUp: “until end of combat” last until end of combat PHASE
phases = "beginning", "precombat main", "combat", "postcombat main", "ending"
beginning_steps = "untap", "upkeep", "draw"
combat_steps = "beginning of combat", "declare attackers", "declare blockers", "combat damage", "end of combat"
ending_steps = "end", "cleanup"

def turn(player, number):
    print(f"{player}`s Turn {number}.")
    print("Beginning: Untap")
    print("Trigger: 'At Beginning of Turn' and 'At Beginning of Untap Step' (go on the stack at beginning of Upkeep)")
    print("Phasing")
    print(f"{player} untaps.")

    print("Beginning: Upkeep")
    print("Trigger: 'At Beginning of Upkeep'")
    priority(player)

    print("Beginning: Draw")
    print("draw(library, 1)")
    print("Trigger: 'At Beginning of Draw Step'")
    priority(player)


    print("Precombat Main")
    priority(player)

    print("Combat: Beginning of Combat")
    priority(player)

    print("Combat: Declare Attackers")
    priority(player)

    print("Combat: Declare Blockers")
    priority(player)

    print("Combat: Combat Damage")
    priority(player)

    print("Combat: End of Combat")
    priority(player)

    print("Postcombat Main")
    priority(player)

    print("Ending: End")
    print("Trigger: 'At Beginning of your end step'")

    print("Ending: Cleanup")

def priority(player):
    """When a player can do something, eg cast spells"""
    print("checking for state-based effects")
    input("Make your move!")


In [ ]:
#TODO work with JSON files
#BUG drawn cards are added as lists inside a list
#BUG sorting the hand doesn't work properly

import random

#the decklist as dictionary (manually compiled)
deck1 = {
"Esper Sentinel":4 ,
"Giver of Runes":4 ,
"Memnite":4 ,
"Ornithopter":4 ,
"Puresteel Paladin":4 ,
"Stoneforge Mystic":4 ,
"Steelshaper's Gift":1 ,
"Colossus Hammer":4 ,
"Shadowspear":1 ,
"Springleaf Drum":3 ,
"Sigarda's Aid":4 ,
"Horizon Canopy":2 ,
"Inkmoth Nexus":4 ,
"Plains":11,
"Silent Clearing":2 ,
"Urza's Saga":4
}

# creating and shuffling the library
library = []
for key in deck1:
    for i in range(deck1[key]):
        library.append(key)
        #library.append(Card.where(name=key).all()) # it takes 88 s to fetch a deck of 60 cards (most likely because all versions of all cards are fetched)
    #print(deck1[i], i)
random.shuffle(library)

"""Draw function
for now it just removes the cards from the library and returns the drawn cards.
Maybe they should be added to hand right away, but I think some cards have conditions before being added to the hand"""
def draw(library, number):
    if number <= len(library):
        drawn_cards = []
        for i in range(number):
            drawn_cards.append(library.pop(0))
        return drawn_cards
    else:
        global gameover
        gameover = 1
        print("You tried to draw more cards than your library contains and therefore loose the game.")
        return 0


#draw starting hand
hand = []
hand.append(draw(library, 7))

gameover = 0
playername = "Alex"

while gameover == 0:
    print(f"{playername}´s library contains {len(library)} cards. Hand: {sorted(hand)}") #TODO sort hand by CMC and type
    useraction = input("What do you want to do? draw - d, concede - c")

    if useraction == "c":
      gameover = 1
      break

    if useraction == "d":
        try:
            carddraw = []
            carddraw = draw(library, int(input("How many cards do you want to draw?")))
            if carddraw == 0:
              break
            hand.append(carddraw)
            if carddraw != None:
                print(f"{playername} draws: {carddraw}")

        except ValueError:
            print("Please enter an integer.")

print("Game ended.")

Alex´s library contains 53 cards. Hand: [['Ornithopter', "Urza's Saga", 'Stoneforge Mystic', "Urza's Saga", 'Puresteel Paladin', 'Plains', 'Inkmoth Nexus']]
What do you want to do? draw - d, concede - cc
Game ended.
